# Organic Neural Network

Create a 3-layer neural network from scratch with python for the MNIST dataset

* Layer 1: inputs
* Layer 2: hidden layer
* Layer 3: outputs

In [29]:
import struct
import numpy as np
import os
from array import array as pyarray
from mnist import MNIST
import matplotlib.pyplot as plt
import scipy

First, let's import the MNIST dataset. There is a python package to translate the page called python-mnist. You need to download the data set as well and direct the "MNIST()" function to where these data sets are located

In [30]:
mndata=MNIST('C:\Users\schultz\Documents\School\ML Class')
[train_images,train_labels]=mndata.load_training()
[test_images,test_labels]=mndata.load_testing()

## Step 1: Create our neural network architecture

* **Layer 1**: 1 input node for each dimension of x
    for the mnist data, there are 784 pixels per image

    The input nodes are transformed by a linear transformation before going to the hidden layer
    $z_1 = x_iW_i + b_i$
    
* **Layer 2**: hidden nodes of a variable number
    
    The hidden nodes are transformed by an activation function within the layer
    $a_2 = activation(z_1)$
    
    The outputs of the activation function are transformed by a linear transformation before going to the output layer
    $z_2 = a_2W_i + b_i$
    
* **Layer 3**: 1 output node for each possible value of the output
    for the mnist data, each picture can be 0-9 so there will be 10 output nodes
    
    The output nodes are transformed by an activation function 
    $a_3 = y = activation(z_2)$



In [31]:
def create_3lnetwork(nn_in,nn_out,nn_hid):
    #nn_in = number of nodes in the input layer
    #nn_out = number of nodes in the output layer
    #nn_hid = number of nodes in the hidden layer

    #we have 2 linear transformation per hidden layer (input to hidden, hidden to output)
    np.random.seed(0)
    #weights of the linear transformation between the input and hidden layer is between 0 and 1
    Wih=.001*np.random.rand(nn_in,nn_hid)
    #bias of the linear transformation between the input and hidden layer can be any value so N(0,1)
    Bih=np.random.randn(1,nn_hid)
    #weights of the linear transformation between the hidden and ouptut layer is between 0 and 1
    Who=.001*np.random.rand(nn_hid,nn_out)
    #bias of the linear transformation between the hidden and output layer can be any value so N(0,1)
    Bho=np.random.randn(1,nn_out)
    
    #return our structure that has been encoded by these weights and biases!
    return Wih,Bih,Who,Bho

--------------------------------------------------------------
---------------------Activation Functions---------------------
--------------------------------------------------------------

Multiple activation functions exist. Below we've encoded a few

In [32]:
def sigmoid_function(z):
    #hidden layer uses sigmoid
    return 1/(1 + np.exp(-z))  

def tanh_function(z):
    return np.tanh(z)

def relu_function(z):       
    return np.maximum(0,z)

def softmax_function(z):
    return np.exp(z)/np.sum(np.exp(z), axis=1, keepdims=True)

--------------------------------------------------------------
--------------Our Final Neural Network Plan-------------------
--------------------------------------------------------------

* **Layer 1**: input x

    $z_1 = xW_ih + b_ih$
    
* **Layer 2**: activation function = relu function

    $a_2 = relu(z_1)$
    
    $z_2 = a_2W_ho + b_ho$
    
* **Layer 3**: activation function = softmax

    $a_3 = y = softmax(z_2)$
----------------------------------------------------------------

## Step 2: Forward Propregation

We apply our transformations to 1 set of inputs to get 1 set of outputs from our network

In [33]:
def forward_prop(inputs_list,Wih,Bih,Who,Bho):
    x=np.reshape(inputs_list,(1,len(inputs_list)))
    hidden_inputs=x.dot(Wih)+Bih
    hidden_outputs=relu_function(hidden_inputs)
    
    final_inputs=hidden_outputs.dot(Who)+Bho
    final_outputs=softmax_function(final_inputs)
    return final_outputs

##  Step 3: Loss Function

We need to compare how well our network did ($y$) to the actual target answer ($t$). This is evaluated through a Loss Function ($\xi$). 
Multiple loss functions exist but we will be using the cross-entropy loss function:

$\xi(t,y) = -\sum_{i=1}^n t_i log(y_i)$

In [34]:
def loss_function(yhat,target):
    return - np.multiply(target,np.log(yhat)).sum()

## Step 4: Back propogation

We need to push the error we experienced between the network output and the actual target answer, currently defined in our loss function, back up the chains to update our network so next time it will predict better

How do we do this? Derivatives!!

Taking the derivative of each layer, we can use the chain rule to influence the weights and biases up to the input layer

For the best and most thorough explination of this, please go read his Vectorization of the backward step:
http://peterroelants.github.io/posts/neural_network_implementation_part04/

-----------------------------------------------------------------
---------------Derivative of Activation Function-----------------
-----------------------------------------------------------------

*Note: All of these derivatives assume we are using the original activation function's output and not its input. 
If using input, for example, the derivative of tanh is 1 - np.multiply(np.tanh(z),np.tanh(z))

In [35]:
def deriv_sigmoid(z):
    return np.multiply(z,(1-z))

def deriv_tanh(z):
    return 1-np.multiply(z,z)

def relu_function(z):       
    return np.maximum(0,z)

def deriv_relu(z):
    z[z<=0]=0
    z[z>0]=1
    return z

In [36]:
def train(inputs_list,targets_list,Wih,Bih,Who,Bho):
    x=np.reshape(inputs_list,(1,len(inputs_list)))
    t=np.reshape(targets_list,(1,len(targets_list)))
    hidden_inputs=x.dot(Wih)+Bih
    hidden_outputs=relu_function(hidden_inputs)
    
    final_inputs=hidden_outputs.dot(Who)+Bho
    final_outputs=softmax_function(final_inputs)

    loss=loss_function(final_outputs,t)
    
#    dL/dz2 = dL/dy * dy/dz2 = Y-T
    dL_dz2=final_outputs-t
#    dz2/da2 = d/da2[a2*W_ho + b_ho] = W_ho
#    dL/da2 = dL/dz2*dz2/da2
    dL_da2=dL_dz2.dot(Who.T)
#    da2/dz1 = d/dz1[activation_function(z1)] = derivative(activation function(hidden_outputs))
    der_act=deriv_relu(hidden_outputs)

#    dz2/dBho = d/dBho[a2*W_ho + b_ho] = 1
#    dL/dBho = dL/dz2*dz2/dBho = dL/dz2*1 = dL/dz2
    dL_Bho=dL_dz2
#    dz2/dWho = d/dWho[a2*W_ho + b_ho] = a2 = hidden_outputs
#    dL/dWho = dL/dz2*dz2/dWho
    dL_Who=hidden_outputs.T.dot(dL_dz2)
    
#    dL/dBih = d/Bih[x*W_ih + b_ih]= 1
#    dL/dBih = dL/da2*da2/dz1 = dL/da2*da2/dz1
    dL_Bih=np.multiply(der_act,dL_da2)
    
#    dL/dWih = d/Wih[x*W_ih + b_ih]= x
#    dL/dWih = dL/da2*da2/dz1*dz1/dWih = dL_Bih*dz1/dWih
    dL_Wih=x.T.dot(dL_Bih)
    
    
    return loss,dL_Who,dL_Bho,dL_Wih,dL_Bih

## Step 5: Update our Original Network Weights

Now that we have by what amount our originals should be updated, we need to update this. 

But how much weight do we put on our 'teaching'? Having one data point shouldn't wipe out our previous calculations (especially when we're 100, 200, or 1000 data points in!)

This is where the learning rate comes in. The learning rate is how much we want our one data point to influence a weight and bias  of our entire system. Typically this is around 0.1-0.0001

Below is what we do for 1 training point:

In [37]:
lr=0.001
target_list=np.zeros(10)
target_list[train_labels[i]]=1
[loss,dL_Who,dL_Bho,dL_Wih,dL_Bih]=train(train_images[i],target_list,Wih,Bih,Who,Bho)
Wih=Wih-(lr*dL_Wih)
Bih=Bih-(lr*dL_Bih)
Who=Who-(lr*dL_Who)
Bho=Bho-(lr*dL_Bho)

NameError: name 'i' is not defined

## Let's train our network!!

We will use all 2000 training points for the mnist data set

In [ ]:
lr=0.001
[Wih,Bih,Who,Bho]=create_3lnetwork(np.shape(train_images)[1],10,200)
for i in range(0,np.shape(train_images)[0]):
    target_list=np.zeros(10)
    target_list[train_labels[i]]=1
    [loss,dL_Who,dL_Bho,dL_Wih,dL_Bih]=train(train_images[i],target_list,Wih,Bih,Who,Bho)
    Wih=Wih-(lr*dL_Wih)
    Bih=Bih-(lr*dL_Bih)
    Who=Who-(lr*dL_Who)
    B=Bho-(lr*dL_Bho)

## Let's see how our trained network does!!

We will use all 2000 test points for the mnist data set and get a percentage of what we correctly predicted

In [ ]:
match=0
for i in range(0,len(test_images)):
    [final_outputs]=forward_prop(test_images[i],Wih,Bih,Who,Bho)
    if np.argmax(final_outputs)==test_labels[i]:
        match=match+1

print("at a learning rate of %f and hidden nodes of %d, %f %% tests matched"%(lr,np.shape(Wih)[1],float(match)/len(test_images)*100))